사용자 기반 필터링 구현하기(작업자 : 권준호)

- 사용자 평점 정보 기반 영화 추천 필터링 
- surprise 사용하기

In [52]:
# !pip install scikit-surprise

In [53]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as SNS
import surprise

In [54]:
# 1. 데이터 가져오기 
# 1-1. import 설정
import pymysql
import pandas as pd


#1-2. DB에서 데이터 가져오기
# inner join을 이용해 movie, genre 테이블 가져오기

conn = pymysql.connect(host='localhost', user='kdigital', password='mysql',
                     db='firstkino', charset='utf8') 
cursor = conn.cursor()

# sql 가져오기
sql = """
    select m.*, g.*, d.*, a.*
    from movie m
    left outer join moviegenre mg
    on m.moviecd = mg.moviecd
    left outer join genre g
    on mg.genrecd = g.genrecd
    left outer join moviedirector md
    on m.moviecd = md.moviecd
    left outer join director d
    on md.directorcd = d.directorcd
    left outer join movieactor ma
    on m.moviecd = ma.moviecd
    left outer join actor a
    on ma.actorcd = a.actorcd
"""

# sql 실행
cursor.execute(query=sql)
rows = cursor.fetchall()

# 결과 가져오기
# for row in rows:
#     print(row)

# 데이터프레임으로 변환하기
movie_content = pd.DataFrame(rows, columns=['movieCd', 'movieNm', 'movieNmEn', 'shortContent', 'prdtYear', 'showTm', 'openDt', 'naverScore', 'imdbScore', 'watchGradeCd', 'genreCd', 'genreNm', 'directorCd', 'directorNm', 'directorNmEn', 'actorCd', 'actorNm', 'actorNmEn'])
movie_content

,movieCd,movieNm,movieNmEn,shortContent,prdtYear,showTm,openDt,naverScore,imdbScore,watchGradeCd,genreCd,genreNm,directorCd,directorNm,directorNmEn,actorCd,actorNm,actorNmEn
0,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,19081,D.W. 그리피스,None,1408,헨리 월설,None
1,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,19081,D.W. 그리피스,None,7525,랄프 루이스,None
2,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,19081,D.W. 그리피스,None,13797,릴리안 기쉬,None
3,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,42546,D.W. 그리피스,None,1408,헨리 월설,None
4,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,42546,D.W. 그리피스,None,7525,랄프 루이스,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520821,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None,7,코미디,40137,최기윤,None,1,,None
520822,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None,19,드라마,16672,최기윤,None,1,,None
520823,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None,19,드라마,40137,최기윤,None,1,,None
520824,20219995,이멀전씨 키트,Emergency Kit,None,2021,None,0,None,None,None,4,기타,16747,전건우,None,1,,None


In [55]:
#1-2. DB에서 데이터 가져오기
# inner join을 이용해 movie, genre 테이블 가져오기

conn = pymysql.connect(host='localhost', user='kdigital', password='mysql',
                     db='firstkino', charset='utf8') 
cursor = conn.cursor()

# sql 가져오기
sql = """
    select * from moviescore
"""

# sql 실행
cursor.execute(query=sql)
rows = cursor.fetchall()

# 결과 가져오기
# for row in rows:
#     print(row)

# 데이터프레임으로 변환하기
userscore = pd.DataFrame(rows, columns=['movieCd','userId', 'userScore'])
userscore = userscore[['userId','movieCd','userScore']]
# userscore = pd.DataFrame(userscore)
# userscore.dtypes
# userscore = userscore.astype({'userId':'str'})
# userscore = userscore.astype({'userScore':'int64'})
# userscore = userscore.astype({'movieCd':'str'})

userscore.to_csv('./userpoint2.csv', index=False, header=False)


In [56]:
type(userscore)

pandas.core.frame.DataFrame

In [57]:
# 2. 데이터 전처리하기
# 2-1. shortContent, genreNm, directNm별로 전처리

movie_content1 = movie_content[['movieCd', 'movieNm', 'actorNm']]
movie_content2 = movie_content[['movieCd', 'movieNm', 'genreNm']]
movie_content3 = movie_content[['movieCd', 'movieNm', 'directorNm']]
movie_content4 = movie_content[['movieCd', 'movieNm', 'shortContent']]

#2-2. 결측치 ""로 대체하기
movie_content1["actorNm"] = movie_content1['actorNm'].fillna("")
movie_content2["genreNm"] = movie_content2['genreNm'].fillna("")
movie_content3["directorNm"] = movie_content3['directorNm'].fillna("")
movie_content4["shortContent"] = movie_content4['shortContent'].fillna("")

C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a

In [58]:
# 2-3. 유사도 비교 컬럼 전처리

import numpy as np

# actor
movie_content1["actorNm"] = movie_content1.groupby(by=["movieCd", "movieNm"])["actorNm"].transform(lambda row: '$$'.join(row))
movie_content1.drop_duplicates(inplace=True)
movie_content1['actorNm'] = movie_content1['actorNm'].map(lambda sc : ','.join(np.unique(sc.split('$$'))))
movie_content1.shape

# genre
movie_content2["genreNm"] = movie_content2.groupby(by=["movieCd", "movieNm"])["genreNm"].transform(lambda row: ','.join(row))
movie_content2.drop_duplicates(inplace=True)
movie_content2['genreNm'] = movie_content2['genreNm'].map(lambda sc : ','.join(np.unique(sc.split(','))))
movie_content2.shape


# director
movie_content3["directorNm"] = movie_content3.groupby(by=["movieCd", "movieNm"])["directorNm"].transform(lambda row: ','.join(row))
movie_content3.drop_duplicates(inplace=True)
movie_content3['directorNm'] = movie_content3['directorNm'].map(lambda sc : ','.join(np.unique(sc.split(','))))
movie_content3.shape

# shortContent
movie_content4["shortContent"] = movie_content4.groupby(by=["movieCd", "movieNm"])["shortContent"].transform(lambda row: ','.join(row))
movie_content4.drop_duplicates(inplace=True)
movie_content4['shortContent'] = movie_content4['shortContent'].map(lambda sc : ','.join(np.unique(sc.split(','))))
movie_content4.shape


C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

(46918, 3)

In [59]:
# 2-4. movieCd, movieNm 기준으로 통합하기
movie_data= movie_content1.merge(movie_content2, on=['movieCd', 'movieNm'], how="inner")\
                                        .merge(movie_content3, on=['movieCd', 'movieNm'], how="inner")\
                                            .merge(movie_content4, on=['movieCd', 'movieNm'], how="inner")

# 통합 데이터 체크
print( movie_data.shape )
type(movie_data)

(46918, 6)


pandas.core.frame.DataFrame

In [60]:
# surprise를 활용한 영화시스템 구현
# 필요한 설정 기능 입력

from surprise.model_selection import train_test_split
from surprise.dataset import DatasetAutoFolds
from surprise.dataset import Dataset, Reader
from surprise import accuracy
from surprise import SVD
import numpy as np

In [61]:
# 3. 훈련 데이터 가져와서 훈련하기

data = Dataset.load_builtin('ml-100k')

# 훈련데이터 나누기
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)




In [62]:
# 4. 잠재 요인 협업 필터링

# SVD를 이용한 잠재요인 협업 필터링 사용
algo = SVD(n_factors=20, random_state=0)
algo.fit(trainset)

# 사용자아이디=uid, 아이템 아이디=iid로 문자열(str)로 입력
uid = str(51)
iid = str(19528001)

# 추천 예측 평점
pred = algo.predict(uid,iid)
pred

# test데이터(전체 데이터의 25%) 추천 예측 평점
predictions = algo.test(testset)

print('prediction type :',type(predictions), ' size:',len(predictions))

predictions[:5]




prediction type : <class 'list'>  size: 25000


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5897229331535616, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.835154683145526, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.116491355975022, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.7448043881897157, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.2857071738917982, details={'was_impossible': False})]

In [63]:
# 4-1. accuracy 모델 성능평가하기

accuracy.rmse(predictions)

RMSE: 0.9456


0.9455519773752545

In [64]:
# 5. 실제 데이터 검증
from surprise.dataset import DatasetAutoFolds

# Reader 객체 생성
reader = Reader(line_format='user item rating', sep=',', rating_scale=(0.5,5))

# DatasetAutoFolds 객체 생성
data_folds = DatasetAutoFolds(ratings_file='./userpoint2.csv', reader=reader)

# 전체데이터를 train으로 지정
trainset = data_folds.build_full_trainset()
trainset

In [65]:
# 5-1. 잠재요인 협업 필터링
algo = SVD(n_epochs=20, n_factors=50, random_state=42)
algo.fit(trainset)

# 사용자 아이디, 아이템 아이디 문자열 입력
uid = str('kwonkai1')
iid = str(19528025)

# 추천 예측 평점
pred_score = algo.predict(uid, iid, verbose=True)

user: kwonkai1   item: 19528025   r_ui = None   est = 3.46   {'was_impossible': False}


In [66]:
# 5-2. userscore 열 추가하기
userscore2 = userscore.copy()
row = userscore2.columns
userscore2.columns = ["userId", "movieCd", "userScore"]
userscore2 = userscore2.append(pd.DataFrame([row.to_list()], columns=["userId", "movieCd", "userScore"]))

# 행렬 내 string 값 제거
userscore2 = userscore2.drop(index=0, axis=0)

In [67]:
# 5-2. 아직 보지않은 영화 데이터 함수
def naversee_surprise(userscore2, movie_data, userId):
    # 특정 user가 평점을 매긴 모든 영화
    seen_movie = userscore2[userscore2['userId']== userId]['movieCd'].astype(int).tolist()

    # 모든 영화명 list 객체 변환하기
    all_movie = movie_data['movieCd'].astype(int).tolist()

    # 보지 않은 영화 리스트 생셩
    naver_seen_movie = [ movie for movie in all_movie if int(movie) not in seen_movie]

    # 일부 정보 출력하기
    all_movie_li = len(all_movie)
    seen_li = len(seen_movie)
    naver_seen_li = len(naver_seen_movie)

    print(f"전체 영화 수: {all_movie_li}, 평점 매긴 영화 수: {seen_li}, 추천 대상 영화 수: {naver_seen_li}")

    return naver_seen_movie

# 실제 유저 평가 영화 보기
naver_seen_movie = naversee_surprise(userscore2, movie_data, 'kwonjh')


전체 영화 수: 46918, 평점 매긴 영화 수: 74, 추천 대상 영화 수: 46844


In [68]:
# 5-3. 실제 유저 평가 예측
def recommend_movie(algo, userId, naver_seen_movie, top_n=20):
    # 아직 보지 않은 영화 예측 평점
    predictions = []
    for movieCd in naver_seen_movie:
        predictions.append(algo.predict(str(userId), str(movieCd)))

    # 리스트 내 predict 객체의 est 기준으로 내림차순 정렬
    def sortkey_est(pred):
        return pred.est

    # key에 리스트 내 객체의 정렬 구조 설정
    predictions.sort(key=sortkey_est, reverse=True)

    # 상위 20개의 prediction 
    top_pred = predictions[:top_n]

    # 영화코드, 제목, 예측 평점 출력
    print(f"Top-{top_n} 추천 영화 리스트")

    for pred in top_pred:
        
        movie_Cd = int(pred.iid)
        movie_title = movie_data[movie_data["movieCd"] == str(movie_Cd)]["movieNm"].tolist()
        movie_rating = pred.est
        
        print(f"{movie_title}: {movie_rating:.2f}")

# 사용자 영화 추천
recommend_movie(algo, 'kwonjh', naver_seen_movie, top_n=20)

Top-20 추천 영화 리스트
['위플래쉬']: 3.84
['캡틴 필립스']: 3.84
['룸']: 3.76
['알라딘']: 3.73
['극장판 포켓몬스터 너로 정했다!']: 3.73
['순정']: 3.73
['비포 선라이즈']: 3.72
['귀향']: 3.72
['이터널 선샤인']: 3.71
['A-특공대']: 3.71
['스파이더맨: 뉴 유니버스']: 3.69
['분노의 질주: 더 익스트림']: 3.68
['오즈의 마법사']: 3.68
['분노의 질주: 더 맥시멈']: 3.68
['토이 스토리 4']: 3.67
['극장판 짱구는 못말려 : 습격!! 외계인 덩덩이']: 3.67
['가부와 메이 이야기']: 3.67
['명탐정 코난:진홍의 연가']: 3.66
['가버나움']: 3.65
['더 플랜']: 3.64


In [69]:
##################################################################3
# 여기까지만

In [70]:
# for idx, data in enumerate(userscore2['movieCd'].tolist()):
#     try:
#         (int(data))
#     except:
#         print(idx, data)

##########################################
